In [ ]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from pathlib import Path
import os
from tqdm import tqdm
import pandas as pd
import sys
import numpy as np
import matplotlib.pyplot as plt
import cv2
import re
import shutil
from pathlib import Path
from sorted_alpha import sorted_alpha
from scipy.spatial import distance
from sklearn.neighbors import NearestNeighbors
from scipy import interpolate
from frame_count import frame_count
from scipy.signal import savgol_filter


In [ ]:
def interpolation (all_frames, detected_frames, track):
    # 1. drop duplicates (needed for interpolation)
    
    gaps_detected_frames = (np.array(detected_frames)[1:]-np.array(detected_frames)[:-1])==1
    gaps_detected_frames = [x for x,y in zip(detected_frames[1:],gaps_detected_frames) if y==False]
    index_gaps = [detected_frames.index(x) for x in gaps_detected_frames]
    #print("gaps:",np.array(gaps_detected_frames)-1, "current frame:", detected_frames[-1], "last frame:",detected_frames[-2])
    index_gaps = [ind for ind in index_gaps if detected_frames[ind]-detected_frames[ind-1]<50]
    insert_track = []
    insert_frames = []
    if len(index_gaps)>0:
        #print('indexes which will be filled:',np.array(index_gaps)-1)
        for ind in index_gaps:

            interp = interpolate.interp1d(detected_frames[ind-1:ind+1], 
                                               track[ind-1:ind+1], kind='slinear', axis=0, assume_sorted=False)
            to_insert_track = interp(np.arange(detected_frames[ind-1]+1,detected_frames[ind]))
            insert_frames.append(np.arange(detected_frames[ind-1]+1,detected_frames[ind]))
         
  
            insert_track.append(to_insert_track)

    else:
        track[-11:] = savgol_filter(track[-11:], 11, 3, axis=0)
        return([track,detected_frames])

    for i,ind in zip(range(len(insert_track)),index_gaps):
        if i==0:
            new_track = np.insert(track, ind, insert_track[i], axis=0)
        else:
            new_track = np.insert(new_track, ind+len(insert_track[i-1]), insert_track[i], axis=0)
            
    for i,ind in zip(range(len(insert_frames)),index_gaps):
        if i==0:
            new_frames = np.insert(detected_frames, ind, insert_frames[i], axis=0)
        else:
            new_frames = np.insert(new_frames, ind+len(insert_frames[i-1]), insert_frames[i], axis=0)
    if len(insert_frames)<11:
        new_track[-11:] = savgol_filter(new_track[-11:], 11, 3, axis=0)
    else:
        new_track[-len(insert_frames):] = savgol_filter(new_track[-len(insert_frames):], 11, 3, axis=0)
    return([new_track.tolist(), new_frames.tolist()])


In [ ]:
def relative_position(track1, track2, new_detection1, new_detection2):
    x_difference = []
    y_difference = []
    for detect1, detect2 in zip(track1[-50:-1], track2[-50:-1]):
        for index,coords in enumerate(zip(detect1, detect2)):
            if index%2 ==0:
                x_difference.append(coords[0]-coords[1])
            else:
                y_difference.append(coords[0]-coords[1])
    if np.mean(np.abs(x_difference)) > np.mean(np.abs(y_difference)): ### do they tend to be farther on the x axis? 
        if np.mean(x_difference) > 0: # is tracklet1 farther on x axis than tracklet2? relative position
            if new_detection1[0] - new_detection2[0] > 0: # is detection 1 farther on x axis than detection 2? i.e. does new detection converge with past history 
                return(1)
            else:
                return(2)
        elif np.mean(x_difference) <0:  # is tracklet2 farther on x axis than tracklet1?
            if new_detection1[0] - new_detection2[0] < 0: # is detection 2 farther on x axis than detection 1? i.e. does new detection converge with past history 
                return(1)
            else:
                return(2)
    elif np.mean(y_difference) > 0:
        if new_detection1[1] - new_detection2[1] > 0: # is detection 1 farther on y axis than detection 2? i.e. does new detection converge with past history 
            return(1)
        else:

            return(2)
    elif np.mean(y_difference) < 0:  # is tracklet2 farther on y axis than tracklet1?
        if new_detection1[1] - new_detection2[1] < 0:
            return(1)
        else:

            return(2)        ### do they tend to be farther on the y axis? 
       

In [ ]:
def get_skeletons(frame_no, detection, track1, track2):
    knn =  NearestNeighbors(n_neighbors=1, metric='euclidean')
    """function that reads a detections frame, checks if necks are present, and returns the 2 closest skeletons from previous detections (in case 3 have been detected)"""
    data = pd.read_csv(detection)
    data = data.fillna(0)
    ## first check if it contains the neck 
    temp_data = data
    for i in range(0, len(data.columns[1:]),3):
        if data[data.columns[1:][i]].loc[1] == 0:
            temp_data = temp_data.drop([data.columns[1:][i],data.columns[1:][i+1], data.columns[1:][i+2]], axis=1) ## drop skeletons with missing neck
    data=temp_data
    if len(data.columns) < 4: ## if not enough skeletons with neck present return 0 
        return(None)
    elif frame_no ==0: 
        if len(data.columns)>4:  ### when first starting tracking initialise with either one or two skeletons 
            neck_1 = [data['x_1'].loc[1], data['y_1'].loc[1]]
            neck_2 = [data['x_2'].loc[1], data['y_2'].loc[1]]
            return([neck_1, neck_2])
        else: 
            neck_1 = [data['x_1'].loc[1], data['y_1'].loc[1]]
            neck_2 = [0,0]
            return([neck_1, neck_2])
    else:
        if len(data.columns)> 10: ### drop the fourth skeleton if it has been detected
            data = data.drop(data.columns[10:], axis=1) 
        ## trying to tell which one is neck 1 and which is neck 2 

        if track2 == []:         ## if track2 isn't initialized yet:
            detections_x = [data[data.columns[1:][col]].loc[1] for col in  range(0, len(data.columns[1:]), 3)]; detections_y = [data[data.columns[2:][col]].loc[1] for col in  range(0, len(data.columns[2:]), 3)]
            detections = np.vstack([detections_x, detections_y]).T
            probs = [data[data.columns[3:][col]].loc[1] for col in  range(0, len(data.columns[3:]), 3)]
            ## fit knn with the current detections and see which is closer to track1.lastitem
            knn.fit(detections); closest_index = knn.kneighbors([track1[-1]])[1];
            neck_1 = detections[closest_index][0][0]
            detections = np.delete(detections, closest_index, axis=0)
            probs = np.delete(probs, closest_index)
            if detections.size !=0:  ## ensuring that if there was just one skeleton there isn't a point in doing any comparisons as neck_2 will be empty
                neck_2 = detections[np.argmax(probs)]
            else:
                neck_2 = [0,0]
            return([neck_1,neck_2])
        else:
            
            detections_x = [data[data.columns[1:][col]].loc[1] for col in  range(0, len(data.columns[1:]), 3)]; detections_y = [data[data.columns[2:][col]].loc[1] for col in  range(0, len(data.columns[2:]), 3)]
            detections = np.vstack([detections_x, detections_y]).T
            
            probs = [data[data.columns[3:][col]].loc[1] for col in  range(0, len(data.columns[3:]), 3)]
            ## fit knn with the current detections and see which is closer to track1.lastitem
            if len(detections)>1:
                knn.fit(detections); closest_index = knn.kneighbors([track1[-1]])[1];
                neck_1 = detections[closest_index[0][0]]
                detections= np.delete(detections, closest_index, axis=0)
                probs = np.delete(probs, closest_index)
                ## get the closest neck to last detection in track2
                knn.fit(detections); closest_index = knn.kneighbors([track2[-1]])[1]
                neck_2 = detections[closest_index[0][0]]
         
                return([neck_1, neck_2])
            else:
                return([detections[0],[0,0]]) 
                

        
    
        

In [ ]:
path = '/Users/andrei-macpro/Documents/Data/meal_openpose/1043_meal'
detects = sorted_alpha(path)

In [ ]:
numbers = [int((x.split("."))[0].split("_")[1]) for x in detects]


In [ ]:
threshold = 200

track1, frames1 = [], []
track2, frames2 = [], []


frames = []
count = 0

# if discrepancy between last el in frames1 and last elem in frames is less than 50 then interpolate

knn =  NearestNeighbors(n_neighbors=1, metric='euclidean')
count = 0
for i,number in zip(range(len(detects)), numbers):
    frames.append(number) 
    print('frame no:',number)
    if i ==0:  ### for the first frame just get the skeletons don't do anything
        necks = get_skeletons(i, os.path.join(path,detects[i]), track1, track2)
        if necks is not None:
            neck1 = np.array(necks[0]) 
            neck2 = np.array(necks[1])
        else: continue
        if neck2[0] > 0:
            
            track1.append(neck1); frames1.append(number)
            track2.append(neck2); frames2.append(number)
        else:
            track1.append(neck1); frames1.append(number)
        continue
    necks = get_skeletons(i, os.path.join(path,detects[i]), track1, track2)
    if necks is not None:neck1 = necks[0]; neck2 = necks[1]; 
    else:continue
    
    if len(track1) < 100 or len(track2)<100:
        ## 1. if only one skeleton has been detected 
 
        if neck2[0]==0: #if new detection only has one skel
            if track2 == []: ## if second tracking hasn't been initialised it
                if distance.euclidean(track1[-1], neck1) < threshold:
                    track1.append(neck1);  frames1.append(number);
                else: track2.append(neck1);  frames2.append(number);
            else:
                knn.fit([track1[-1]]); distance1 = knn.kneighbors([neck1])[0][0]
                knn.fit([track2[-1]]); distance2 = knn.kneighbors([neck2])[0][0]
                if distance1>distance2:
                    track1.append(neck1); frames1.append(number); 
                else:
                    track2.append(neck1); frames2.append(number); 
        else: ## if two skeletons have been detected
            if track2 == []: ### if second tracking hasn't been initialised it
                knn.fit([track1[-1]]); distance1 = knn.kneighbors([neck1])[0][0];  #  
                distance2 = knn.kneighbors([neck2])[0][0]
                track1.append([neck1,neck2][np.argmin([distance1,distance2])]);frames1.append(number); 
                track2.append([neck1,neck2][np.argmax([distance1,distance2])]); frames2.append(number);
            else:
                knn.fit([track1[-1], track2[-1]]); ## if both tracks have been initialized 
                neighb1 = knn.kneighbors([neck1])
                neighb2 = knn.kneighbors([neck2])
                if neighb1[1][0] == 0 and neighb2[1][0]==1:
                    track1.append(neck1); frames1.append(number); 
                    track2.append(neck2); frames2.append(number); 
                elif neighb1[1][0] == 1 and neighb2[1][0]==0:
                    track1.append(neck2) ;frames1.append(number); 
                    track2.append(neck1) ;frames2.append(number); 
                         
    elif neck2[0]==0: #if new detection only has one skel
                if track2 == []: ## if second tracking hasn't been initialised it
                    if distance.euclidean(track1[-1], neck1) < threshold:
                        track1.append(neck1);  frames1.append(number); 
                    else:track2.append(neck1);  frames2.append(number)
                else:
                 
                    knn.fit([track1[-1]]); distance1 = knn.kneighbors([neck1])[0][0]  #  
                    knn.fit([track2[-1]]); distance2 = knn.kneighbors([neck1])[0][0]
                    if distance1<distance2:
                        track1.append(neck1); frames1.append(number); 
                    else:
                        track2.append(neck1); frames2.append(number);
    elif distance.euclidean(neck1, neck2) < threshold: ## if they are very close together
        rel_position = relative_position(track1, track2, neck1, neck2)
        if rel_position == 1:
            track1.append(neck1); frames1.append(number); 
            track2.append(neck2); frames2.append(number); 
        else:
            track1.append(neck2) ; frames1.append(number); 
            track2.append(neck1); frames2.append(number); 
                           
    
    else:  ### if they aren't close just do location based tracking
  
        knn.fit([track1[-1], track2[-1]]);
        neighb1 =  knn.kneighbors([neck1])
        neighb2 = knn.kneighbors([neck2])

        
        if neighb1[1][0] == 0 and neighb2[1][0]==1:
            track1.append(neck1); frames1.append(number); 
            track2.append(neck2); frames2.append(number);
        elif neighb1[1][0] == 1 and neighb2[1][0]==0:
            track1.append(neck2); frames1.append(number); 
            track2.append(neck1); frames2.append(number);
        else:
            continue
    

    if len(frames1)>11 and frames1[-1]-frames1[-2]>1:
            fill = interpolation(frames, frames1, track1)
            frames1 = fill[1]
            track1 = fill[0]
            
    elif len(frames1)>11 and frames1[-1]-frames1[-12]>10 :
         track1[-11:] = savgol_filter(track1[-11:], 11, 3, axis=0).tolist()
            
    if len(frames2)>11 and frames2[-1]-frames2[-2]>1 :
                fill = interpolation(frames, frames2, track2)
                frames2 = fill[1]
                track2 = fill[0]
    elif len(frames2)>11 and frames2[-1]-frames2[-12]>10 :
        track2[-11:] = savgol_filter(track2[-11:], 11, 3, axis=0).tolist()

   
    
    
print('success')


In [ ]:
len(track2)

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(7, 7))
axs.set_title("Signal")
axs.plot(track1, color='C0')
axs.set_xlabel("Time")
axs.set_ylabel("Amplitude")

In [ ]:
len(track1), len(frames1), len(track2), len(frames2) 

In [ ]:
data = pd.DataFrame([[x[0] for x in track1], [x[1] for x in track1],frames1]).T
data.columns = ['x','y', 'frame']
data

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(7, 7))
axs.set_title("Signal")
axs.plot(frames2, track2, color='C0')
axs.set_xlabel("Time")
axs.set_ylabel("Amplitude")

In [ ]:
frames_video = frame_count('/Users/andrei-macpro/Documents/Data/Meal/1043_meal.mp4', manual=True)

In [ ]:
radius = 20
color1 = (0, 0, 255)
color2 = (255,0,0)



In [ ]:
cap = cv2.VideoCapture('/Users/andrei-macpro/Documents/Data/Meal/1043_meal.mp4')
fourcc = cv2.VideoWriter_fourcc(*'mp4v')# note the lower case
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH) 
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT) 
out = cv2.VideoWriter('1043_meal_tracking.mp4', fourcc, 25.0, (int(width),int(height)))

In [ ]:


for detection in numbers: 
    if detection in set(frames1) and detection in set(frames2):
        cap.set(1, detection)
        ret, image = cap.read()
        image = cv2.circle(image, tuple([int(x) for x in track1[frames1.index(detection)]]), radius=20, color=color1, thickness=-1)
        image = cv2.circle(image, tuple([int(x) for x in track2[frames2.index(detection)]]), radius=20, color=color2, thickness=-1)
        out.write(image)
      
   
    elif detection in set(frames1):
        cap.set(1, detection)
        ret, image = cap.read()
        image = cv2.circle(image, tuple([int(x) for x in track1[frames1.index(detection)]]), radius=20, color=color1, thickness=-1)
  
        out.write(image)

    elif detection in set(frames2):
        cap.set(1, detection)
        ret, image = cap.read()
        image = cv2.circle(image, tuple([int(x) for x in track2[frames2.index(detection)]]), radius=20, color=color2, thickness=-1)

        out.write(image)
 
cap.release()
out.release()
cv2.destroyAllWindows()   
    

In [ ]:
track1

In [ ]:
!pwd

# now for all 

In [ ]:
path = '/Users/andrei-macpro/Documents/Data/play_openpose'
folders = sorted_alpha(path)

In [ ]:
folders[69:]

In [ ]:
threshold = 200


for folder in tqdm(folders[69:]):
    new_path = '/Users/andrei-macpro/Documents/Data/play_tracking'
    os.mkdir(os.path.join(new_path, folder))
    os.chdir(os.path.join(path,folder))
    detects = sorted_alpha(os.path.join(path,folder))
    numbers = [int((x.split("."))[0].split("_")[1]) for x in detects]
    
    track1, frames1 = [], []
    track2, frames2 = [], []
    frames = []
    
    # if discrepancy between last el in frames1 and last elem in frames is less than 50 then interpolate
    
    knn =  NearestNeighbors(n_neighbors=1, metric='euclidean')
        
    for i,number in zip(range(len(detects)), numbers):
    
        frames.append(number) 
        if i ==0:  ### for the first frame just get the skeletons don't do anything
            necks = get_skeletons(i, os.path.join(path,folder,detects[i]), track1, track2)
            if necks is not None:
                neck1 = np.array(necks[0]) 
                neck2 = np.array(necks[1])
            else: continue
            if neck2[0] > 0:
                
                track1.append(neck1); frames1.append(number)
                track2.append(neck2); frames2.append(number)
            else:
                track1.append(neck1); frames1.append(number)
            continue
        necks = get_skeletons(i, os.path.join(path,folder,detects[i]), track1, track2)
        if necks is not None:neck1 = necks[0]; neck2 = necks[1]; 
        else:continue
        
        if len(track1) < 100 or len(track2)<100:
            ## 1. if only one skeleton has been detected 
     
            if neck2[0]==0: #if new detection only has one skel
                if track2 == []: ## if second tracking hasn't been initialised it
                    if distance.euclidean(track1[-1], neck1) < threshold:
                        track1.append(neck1);  frames1.append(number);
                    else: track2.append(neck1);  frames2.append(number);
                else:
                    knn.fit([track1[-1]]); distance1 = knn.kneighbors([neck1])[0][0]
                    knn.fit([track2[-1]]); distance2 = knn.kneighbors([neck2])[0][0]
                    if distance1>distance2:
                        track1.append(neck1); frames1.append(number); 
                    else:
                        track2.append(neck1); frames2.append(number); 
            else: ## if two skeletons have been detected
                if track2 == []: ### if second tracking hasn't been initialised it
                    knn.fit([track1[-1]]); distance1 = knn.kneighbors([neck1])[0][0];  #  
                    distance2 = knn.kneighbors([neck2])[0][0]
                    track1.append([neck1,neck2][np.argmin([distance1,distance2])]);frames1.append(number); 
                    track2.append([neck1,neck2][np.argmax([distance1,distance2])]); frames2.append(number);
                else:
                    knn.fit([track1[-1], track2[-1]]); ## if both tracks have been initialized 
                    neighb1 = knn.kneighbors([neck1])
                    neighb2 = knn.kneighbors([neck2])
                    if neighb1[1][0] == 0 and neighb2[1][0]==1:
                        track1.append(neck1); frames1.append(number); 
                        track2.append(neck2); frames2.append(number); 
                    elif neighb1[1][0] == 1 and neighb2[1][0]==0:
                        track1.append(neck2) ;frames1.append(number); 
                        track2.append(neck1) ;frames2.append(number); 
                             
        elif neck2[0]==0: #if new detection only has one skel
                    if track2 == []: ## if second tracking hasn't been initialised it
                        if distance.euclidean(track1[-1], neck1) < threshold:
                            track1.append(neck1);  frames1.append(number); 
                        else:track2.append(neck1);  frames2.append(number)
                    else:
                     
                        knn.fit([track1[-1]]); distance1 = knn.kneighbors([neck1])[0][0]  #  
                        knn.fit([track2[-1]]); distance2 = knn.kneighbors([neck1])[0][0]
                        if distance1<distance2:
                            track1.append(neck1); frames1.append(number); 
                        else:
                            track2.append(neck1); frames2.append(number);
        elif distance.euclidean(neck1, neck2) < threshold: ## if they are very close together
            rel_position = relative_position(track1, track2, neck1, neck2)
            if rel_position == 1:
                track1.append(neck1); frames1.append(number); 
                track2.append(neck2); frames2.append(number); 
            else:
                track1.append(neck2) ; frames1.append(number); 
                track2.append(neck1); frames2.append(number); 
                               
        
        else:  ### if they aren't close just do location based tracking
      
            knn.fit([track1[-1], track2[-1]]);
            neighb1 =  knn.kneighbors([neck1])
            neighb2 = knn.kneighbors([neck2])
    
            
            if neighb1[1][0] == 0 and neighb2[1][0]==1:
                track1.append(neck1); frames1.append(number); 
                track2.append(neck2); frames2.append(number);
            elif neighb1[1][0] == 1 and neighb2[1][0]==0:
                track1.append(neck2); frames1.append(number); 
                track2.append(neck1); frames2.append(number);
            else:
                continue
        
    
        if len(frames1)>11 and frames1[-1]-frames1[-2]>1:
            fill = interpolation(frames, frames1, track1)
            frames1 = fill[1]
            track1 = fill[0]
            
        elif len(frames1)>11 and frames1[-1]-frames1[-12]>10 :
             track1[-11:] = savgol_filter(track1[-11:], 11, 3, axis=0).tolist()

        if len(frames2)>11 and frames2[-1]-frames2[-2]>1 :
                    fill = interpolation(frames, frames2, track2)
                    frames2 = fill[1]
                    track2 = fill[0]
        elif len(frames2)>11 and frames2[-1]-frames2[-12]>10 :
            track2[-11:] = savgol_filter(track2[-11:], 11, 3, axis=0).tolist()
    data1 = pd.DataFrame([[x[0] for x in track1], [x[1] for x in track1],frames1]).T
    data1.columns = ['x','y', 'frame']
    data1.set_index('frame', inplace=True)
    data1.to_csv(os.path.join(new_path, folder, 'track1.csv'))
    data2 = pd.DataFrame([[x[0] for x in track2], [x[1] for x in track2],frames2]).T
    data2.columns = ['x','y', 'frame']
    data2.set_index('frame', inplace=True)
    data2.to_csv(os.path.join(new_path, folder, 'track2.csv'))       

In [ ]:
folders

In [ ]:
print('success')